In [1]:
# Import necessary libraries
import pandas as pd
import plotly.graph_objects as go
import dalmatian
from google.cloud import storage
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

import numpy as np

import sys
import os
sys.path.append('..')  
import model_helper 
from helper import *
from plotly_helper import *

In [2]:
# Function to load data
def load_data(patient_id, wbc_file, treatment_file):
    # Load WBC and treatment data
    wbc_df = pd.read_csv(wbc_file)
    treatment_df = pd.read_csv(treatment_file, sep='\t')
    
    # Get input files from Terra through dalmatian
    workspace = 'broad-firecloud-ibmwatson/TAG_CLL_Clonal_Kinetic_UMI_PrAN'
    wm = dalmatian.WorkspaceManager(workspace)
    participants = wm.get_participants()
    
    # Load additional data for the patient
    cluster_CCF_df = pd.read_csv(participants.loc[patient_id]['cluster_ccfs'], sep='\t')
    
    tree_df = pd.read_csv(f"../Cell_Population/{patient_id}_build_tree_posteriors.tsv", sep='\t')
    
    
    abundance_file = f"../Cell_Population/{patient_id}_cell_population_abundances.tsv"
    mcmc_file = f"../Cell_Population/{patient_id}_cell_population_mcmc_trace.tsv"  
    
    abundance_df = pd.read_csv(abundance_file, sep='\t')
    mcmc_df = pd.read_csv(mcmc_file, sep='\t')  
    
    return wbc_df, treatment_df, cluster_CCF_df, abundance_df, mcmc_df, tree_df

# Function to filter data for a specific patient
def filter_patient_data(wbc_df, patient_id):
    wbc_df_patient = wbc_df[wbc_df['Patient'] == patient_id]
    wbc_df_patient.reset_index(drop=True, inplace=True)
    times_sample = [int(i) for i in wbc_df_patient.loc[wbc_df_patient.Sample.notna(), 'Time_since_start_tx'].tolist()]
    wbc_all = [float(i) for i in wbc_df_patient['WBC'].tolist()]
    
    
    CLL_count_sample = [float(i) for i in wbc_df_patient.loc[wbc_df_patient.Sample.notna(), 'CLL count estm'].tolist()]
    all_times = [int(i) for i in wbc_df_patient['Time_since_start_tx'].to_list()]
    
    return wbc_df_patient, times_sample, CLL_count_sample, wbc_all,all_times





In [11]:
end_treatment_df = pd.read_csv("../CLL8_end_treatment.csv")

In [5]:
# Input patient information
# Input patient information
patient_id = 'GCLL-0105'
wbc_file = '../CLL8 CLL cell count 05222025 UPDATED.csv'
treatment_file = '../GCLL_treatment.txt'
UMI_start = 1
UMI_end = 3
treatment_start = 0


In [6]:
# Manually verify the sample list order is correct

In [7]:

sample_list = ['GCLL-0105-T-01', 'RP-1895_PDO-33508_SM-NKK1R_GCLL-0105-MRD-01_v1_Custom_Selection_OnPrem', 'RP-1895_PDO-34670_SM-NPXSO_GCLL-0105-MRD-02_v1_Custom_Selection_OnPrem', 'GCLL-0105-T-02']

In [63]:
# Output file

In [8]:
outputfile_name = "CLL_subplot_combined_0105_wbc_updated.html"

Update patient specific information in the below cell 

In [12]:
# Get the treatment duration 

treatment_end = end_treatment_df[end_treatment_df.Patient == patient_id].Treatment_duration.iloc[0]

In [13]:
treatment_end

176

In [14]:

wbc_df, treatment_df, cluster_CCF_df, abundance_df, mcmc_df, tree_df = load_data(patient_id, wbc_file,treatment_file )

# Filter data for the specific patient
wbc_df_patient, times_sample, CLL_count_sample,wbc_count, all_times = filter_patient_data(wbc_df, patient_id)

# Plot CLL count over time and save as HTML
CLL_plot_html = plot_CLL_count(patient_id, times_sample, CLL_count_sample, UMI_start, UMI_end, treatment_start, treatment_end, )
    




In [15]:
times_sample

[-19, 1372, 1560, 2304]

In [16]:
wbc_count

[107.78, 2.4, 3.0, 3.5, 5.2, 2.9, 3.7, 4.3, 3.8, 4.4, 68.9]

In [17]:
CLL_count_sample


[81.9128, 0.01394894, 0.15416605, 62.01]

In [18]:
wbc_table_html = plot_metadata_table(wbc_df_patient, patient_id )

In [19]:
tree_df

,n_iter,likelihood,edges
0,51,"[0.3333333333333341, 0.25000000000000083, 0.49...","1-2,1-4,4-5,4-6,1-3,None-1"
1,47,"[0.25000000000000083, 0.3333333332604348, 0.25...","1-2,1-3,2-4,2-5,3-6,None-1"
2,39,"[0.25000000000000083, 0.4999999741374485, 0.49...","1-2,1-3,2-4,3-6,4-5,None-1"
3,27,"[0.2499999967821642, 0.24999999725951036, 0.19...","1-2,1-3,2-4,4-5,4-6,None-1"
4,27,"[0.3333333333333341, 0.25000000000000083, 0.33...","1-2,1-4,2-5,4-3,4-6,None-1"
5,15,"[0.49999999999999906, 0.49999999999999906, 0.4...","1-2,1-4,3-6,4-3,4-5,None-1"
6,14,"[0.19999999999999943, 0.24999999982187843, 0.4...","1-2,1-3,2-4,2-6,4-5,None-1"
7,13,"[0.3333333333333341, 0.3333333333333341, 0.499...","1-2,1-3,2-4,2-5,4-6,None-1"
8,11,"[0.25000000000000083, 0.25000000000000083, 0.4...","1-2,1-4,2-5,2-6,4-3,None-1"
9,5,"[0.3333333333333341, 0.49999999999999906, 0.49...","1-2,1-4,2-6,4-3,4-5,None-1"


In [20]:
 plot_tree_html = plot_tree_plotly(tree_df, 0)

In [21]:
 plot_ccf_html = plot_ccf(cluster_CCF_df, times_sample, treatment_df,)

In [46]:
branch_annotations = {
    1: "del(14q32)",
    
    3: "SAMHD1_p.Q571* , SAMHD1_p.M216fs",
    4: "POT1_p.Q94R, RRM1_p.N85I,TLR4_p.I673S",
    5: "POT1_p.D42E, CDC25B_p.A454E ",
    6: "NOTCH1_p.A2043T, KMT2C_p.R3079* "
    
}

In [47]:
ccf_tree_html = plot_ccf_tree_combined(
    tree_df=tree_df,
    tree_selected=0,
    ccf_df=cluster_CCF_df,
    times_sample=times_sample,
    treatment_df=treatment_df,
    branch_annotations=branch_annotations

)

SAMHD1_p.Q571*<br>SAMHD1_p.M216fs
POT1_p.Q94R<br>RRM1_p.N85I<br>TLR4_p.I673S
POT1_p.D42E<br>CDC25B_p.A454E
NOTCH1_p.A2043T<br>KMT2C_p.R3079*


In [24]:
cluster_list, cluster_abundance = model_helper.get_abundance(abundance_df, mcmc_df, sample_list)
subclone_sample, log_subclone_sample = model_helper.calc_subclone(CLL_count_sample, cluster_abundance, cluster_list)
all_abundance = model_helper.get_all_abundance(cluster_list, mcmc_df, sample_list, times_sample, )

In [25]:
#  Add noise to all abundance

import random 
random.seed(42)
def add_uniform_noise_and_normalize(lst, low=0, high=0.01):
    
    # Add uniform noise
    noisy_lst = [x + random.uniform(low, high) for x in lst]

    
    # Normalize the list
    total_sum = sum(noisy_lst)
    
    normalized_lst = [x / total_sum for x in noisy_lst]

    return normalized_lst


subclone_cluster_iter = {}
# Iterate through each cluster
for cluster, iterations in all_abundance.items():
   
    # Iterate through each iteration in the cluster
    for iteration, value in iterations.items():
        # If the iteration is not in the new dict, add it
        if iteration not in subclone_cluster_iter:
            subclone_cluster_iter[iteration] = {}
        # Add the cluster and its value to this iteration
        subclone_cluster_iter[iteration][cluster] = value

        
        
noise_added_iter = {}

for iteration, abundances in subclone_cluster_iter.items():
    
    transposed_data = {i: list(values) for i, values in enumerate(zip(*abundances.values()), start=1)}
    
    noise_added_iter[iteration] = {}
    for time_point, abundance in transposed_data.items():
        
#         print(time_point, abundance)
        noise_abundance = add_uniform_noise_and_normalize(abundance)
    
#         print(noise_abundance)

        noise_added_iter[iteration][time_point] = add_uniform_noise_and_normalize(abundance)

    
noise_added_all_abundance = {}
for iteration, abundances in noise_added_iter.items():
    original_format = {i + 1: list(values) for i, values in enumerate(zip(*abundances.values()))}
    noise_added_all_abundance[iteration] = original_format
    
    
noise_added_original_all_abundances = {}

for iteration, abundances in noise_added_all_abundance.items():
    
    for cluster, abundance_value in abundances.items():
        if cluster not in noise_added_original_all_abundances:
            noise_added_original_all_abundances[cluster] = {}
        noise_added_original_all_abundances[cluster][iteration] = abundance_value
            

In [26]:
subclone_sample_mcmc_with_uniform_noise, log_subclone_sample_mcmc_with_uniform_noise = model_helper.calc_subclone(CLL_count_sample, noise_added_original_all_abundances, cluster_list, input_type = "mcmc")

In [27]:
times_aft_tx = [x for x in all_times if x > treatment_end]
times_aft_tx.insert(0, treatment_end)

# Set the extrapolate time after treatment
extrapolate_start_idx = 1

In [28]:
times_sample

[-19, 1372, 1560, 2304]

In [29]:
times_sample[extrapolate_start_idx:]

[1372, 1560, 2304]

In [30]:
# Generate Plotly plot
subclone_plot_html = plot_subclones(cluster_list, times_sample, CLL_count_sample, log_subclone_sample, extrapolate_start_idx, times_aft_tx, treatment_df, treatment_end)


In [31]:
linear_model_mcmc_html = plot_linear_model_mcmc(cluster_list,times_sample,  CLL_count_sample, log_subclone_sample_mcmc_with_uniform_noise,extrapolate_start_idx,times_aft_tx,treatment_df, treatment_end)  

In [32]:
# Convert the column "CLL count estm" to float 
wbc_df_patient['CLL count estm'] = wbc_df_patient['CLL count estm'].astype(float) 
times_sliced_aft = [int(i) for i in wbc_df_patient[wbc_df_patient['CLL count estm'] > 0]['Time_since_start_tx'].values  if int(i) > 0]

/var/folders/x6/7zg8fddx4dlgx2lv4sv34_g40000gp/T/ipykernel_36156/981427378.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [33]:
times_aft_tx

[176, 379, 484, 575, 673, 758, 1288, 1372, 1560, 2304]

In [34]:
# Needs to manually check times_sliced_aft

In [35]:
times_sliced_aft

[1288, 1372, 1560, 2304]

In [36]:
len(times_sliced_aft)

4

In [37]:
index_samples_model = slice(1,4)

# index_samples_model is the index of the samples after treatment. For instance, slice(1,7) means sample 2, 3, 4, 5,6, 7

X, y = create_inputs(times_sliced_aft, log_subclone_sample_mcmc_with_uniform_noise,2, index_samples_model, times_sample)



In [38]:
y

[None,
 array([ -8.84806311,  -5.5933742 ,  -4.61839104,  -9.20041558,
         -9.24779195, -10.58786467]),
 array([-9.99256025, -2.70531079, -3.47494339, -3.5835603 , -4.13125792,
        -4.37720706]),
 array([ 1.25548447,  2.59908869,  3.63670046, -1.07803581, -1.78826466,
         1.88893487])]

In [39]:
wbc_aft_tx = wbc_df_patient[wbc_df_patient.Time_since_start_tx > treatment_end]['CLL count estm'].to_list()
wbc_model = [i for i in wbc_aft_tx if i > 0]

In [40]:
wbc_model

[0.01369249, 0.01394894, 0.15416605, 62.01]

In [41]:
len(wbc_model)

4

In [42]:
n_clusters = len(cluster_list)
logsumexp_points = np.log(wbc_model)
model = MultiClusterLinearRegression(n_clusters, X, y)
model.fit(logsumexp_points)

In [43]:
# Generate Plotly plot
subclone_plot_new_model_html = plot_subclones_new_model(cluster_list, times_sample, wbc_model, log_subclone_sample, extrapolate_start_idx, times_aft_tx, times_sliced_aft, treatment_df,treatment_end, model )


In [44]:
plot_mcmc_new_model_html = plot_mcmc_model(cluster_list, index_samples_model, times_aft_tx, times_sample, times_sliced_aft, sample_list, wbc_model, log_subclone_sample_mcmc_with_uniform_noise, treatment_df, treatment_end)


In [48]:
create_html_file([wbc_table_html, plot_ccf_html, ccf_tree_html, CLL_plot_html, subclone_plot_html,linear_model_mcmc_html,subclone_plot_new_model_html, plot_mcmc_new_model_html], output_file=outputfile_name)

HTML file saved as CLL_subplot_combined_0105_wbc_updated.html
